In [1]:
NO_PARTITIONS = 10

def partition_dataset(qids, X, y):
	N_queries = qids.size
	chunks_q = []
	chunks_data = []
	sm = 0
	for i in range(NO_PARTITIONS):
	    if i == NO_PARTITIONS - 1:
	        nr = N_queries - sum(chunks_q)
	    else:
	        nr = N_queries // NO_PARTITIONS
	    chunks_q.append(nr)
	    chunks_data.append(np.sum(qids[sm:sm+nr]))
	    sm += nr
	    
	chunks_data = dask.compute(chunks_data)[0]
	# print('Chunks for qids', chunks_q)
	# print('Chunks for data', chunks_data)

	dx = X.to_dask_array(lengths=True)
	dy = y.to_dask_array(lengths=True)

	qids = qids.rechunk([tuple(chunks_q)])
	dx = dx.rechunk([tuple(chunks_data), (136,)])
	dy = dy.rechunk([tuple(chunks_data)])

	return dx, dy, qids

In [5]:
import dask
from dask.distributed import Client, LocalCluster, wait
from dask.dataframe import read_csv
from lightgbm import DaskLGBMRanker
from sklearn.model_selection import GroupKFold
import numpy as np
from lightgbm import DaskLGBMRanker
from copy import deepcopy
import warnings

cluster = LocalCluster(n_workers=1)
client = Client(cluster)
print('[0/3] Created dashboard at', cluster.dashboard_link)

df = read_csv('train_mock.csv')
df['QID'] = df['QID'] + 5

print('[1/3] Finished data reading ...')

from dask_ml import preprocessing
print(df.head(50))
le = preprocessing.LabelEncoder()
df['QID'] = le.fit_transform(df['QID'])
print(df.head(50))

# qids = train_df.groupby("QID")["QID"].count().to_dask_array(lengths=True)
# X = train_df.drop(["QID", "Relevance"], axis=1)
# y = train_df["Relevance"]

# dx, dy, qids = partition_dataset(qids, X, y)
# groups = train_df['QID'].to_dask_array(lengths=True).rechunk(dy.chunks)

# print('[2/3] Finished creating train dataset ...')

c:\users\silviu\appdata\local\programs\python\python38\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53265 instead
  warnings.warn(


[0/3] Created dashboard at http://127.0.0.1:53265/status
[1/3] Finished data reading ...
    Relevance  QID  Feature 1  Feature 2  Feature 3  Feature 4  Feature 5  \
0           3    5   0.318192   0.607159   0.725744   0.629238   0.273241   
1           4    5   0.173313   0.836317   0.534525   0.764702   0.102095   
2           2    5   0.991165   0.600911   0.454357   0.019018   0.053494   
3           3    5   0.594217   0.362520   0.454515   0.729441   0.939580   
4           0    5   0.429736   0.424988   0.680221   0.666797   0.947421   
5           2    5   0.365438   0.352250   0.374603   0.411588   0.506876   
6           1    5   0.781077   0.233929   0.432393   0.811549   0.961533   
7           3    5   0.450699   0.286126   0.185567   0.524982   0.046645   
8           2    5   0.386208   0.316384   0.538216   0.099309   0.545726   
9           0    5   0.776577   0.191949   0.969499   0.887779   0.513105   
10          4    6   0.797448   0.087818   0.028450   0.300121  

    Relevance  QID  Feature 1  Feature 2  Feature 3  Feature 4  Feature 5  \
0           3    0   0.318192   0.607159   0.725744   0.629238   0.273241   
1           4    0   0.173313   0.836317   0.534525   0.764702   0.102095   
2           2    0   0.991165   0.600911   0.454357   0.019018   0.053494   
3           3    0   0.594217   0.362520   0.454515   0.729441   0.939580   
4           0    0   0.429736   0.424988   0.680221   0.666797   0.947421   
5           2    0   0.365438   0.352250   0.374603   0.411588   0.506876   
6           1    0   0.781077   0.233929   0.432393   0.811549   0.961533   
7           3    0   0.450699   0.286126   0.185567   0.524982   0.046645   
8           2    0   0.386208   0.316384   0.538216   0.099309   0.545726   
9           0    0   0.776577   0.191949   0.969499   0.887779   0.513105   
10          4    1   0.797448   0.087818   0.028450   0.300121   0.885880   
11          2    1   0.498895   0.743142   0.645731   0.566611   0.540252   

In [5]:
gkf = GroupKFold(n_splits=5)
for train_index, valid_index in gkf.split(dx, dy, groups):
    print(train_index)
    break

[     86      87      88 ... 3017511 3017512 3017513]


In [15]:
g1 = groups[train_index].compute()
g2 = groups[valid_index].compute()

In [25]:
g1 = dask.array.unique(g1).compute()
g2 = dask.array.unique(g2).compute()

In [44]:
test_df = read_csv('test.csv')

In [48]:
test_df['QID'].head(120)

0       5
1       5
2       5
3       5
4       5
       ..
115    10
116    10
117    10
118    10
119    10
Name: QID, Length: 120, dtype: int64

In [9]:
dask.array.unique(groups[train_index]).compute()

array([    1,     2,     3, ..., 31527, 31528, 31530], dtype=int64)